# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
import re
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet') # download for lemmatization
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import numpy as np
from nltk.stem.wordnet import WordNetLemmatizer
from sqlalchemy import create_engine
import pandas as pd
from itertools import chain

from sklearn.datasets import make_multilabel_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///Anand_ETL_Pipeline_16022020.db')
df = pd.read_sql_table('Cyclone',engine)
X = df.message.values

y = df.iloc[:,4:40]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    words = []
    tokens = []

    #normalize
    for i in range(len(text)):
        #print(text[i],"in text[i]")
        if (type(text[i]).__name__ == 'list'):
            for j in range(len(text[i])):
                #print(text[j],"in text[j]")
                #words = [re.sub(r"[^a-zA-Z]", " ", text[i][j].lower())]
                words.append([re.sub(r"[^a-zA-Z]", " ", text[i][j].lower())])
        else:         
            words.append(re.sub(r"[^a-zA-Z]", " ", text[i].lower()))
            #print(text[i],"in text[i] in else")
            #words = re.sub(r"[^a-zA-Z]", " ", text[i].lower())

            
            
    #tokenize        
    for i in range(len(words)):
        if (type(words[i]).__name__ == 'list'):
            for j in range(len(words[i])):
                tokens.append(word_tokenize(words[i][j]))
        else:         
            tokens.append(word_tokenize(words[i]))
            #print(text[i],"in text[i] in else")
            #words = re.sub(r"[^a-zA-Z]", " ", text[i].lower())

            
    
    # putting all tokens in one list rather than leaving them list of lists
    tokens = list(chain(*tokens))
    
    #remove stop words
    tokens = [t for t in tokens if t not in stopwords.words("english")]
    
    #remove duplicate tokens from the list
    #tokens = list(set(tokens))
    
    #lemmatize
    # Reduce words to their root form
    lemmed_before = [WordNetLemmatizer().lemmatize(w) for w in tokens]
    
    # Lemmatize verbs by specifying pos
    lemmed_after = [WordNetLemmatizer().lemmatize(w, pos='v') for w in lemmed_before]

    return lemmed_after


#token = lambda x: tokenize(x)

#X = [["For today:= this is, a  advanced _ example #- String!"],["For today:= this is, a  advancedtwo _ example #- String!"],["For today:= this is, a 999 advancedthree _ example #- String!"],["For today:= this is, a 999 advancedfour _ example #- String!"],"For today:= this is, a 999 advancedfive _ example #- String!"]

#tokenize(X)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
# build pipeline
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier())
])


### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
#Spliting into Training and Test data sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=20)

###using 1% of actual training set for troubleshooting
#X_train = X_train[:len(X_train)/100]
#y_train = y_train[:len(y_train)/100]

#print(X_train, y_train, X_test, y_test)

# train classifier
pipeline.fit(X_train,y_train)

print(X_train.size, y_train.size, X_test.size, y_test.size)

20972 754992 5244 188784


In [9]:
y_pred = pipeline.predict(X_test)

# 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [11]:
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

#test_size = 0.33
#seed = 7

#X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, y, test_size=test_size, random_state=seed)
#X_traincr, y_traincr , X_testcr, y_testcr = model_selection.train_test_split(X, y, test_size=test_size, random_state=seed)

#model = LogisticRegression()
#model.fit(X_traincr, y_traincr)

#predicted = model.predict(X_testcr)

target_names = df.iloc[:,4:40].columns

print(X.shape)
print(y.shape)

X_train = X_train.astype(str)
X_test = X_test.astype(str)
y_train = y_train.astype(str)
y_test = y_test.astype(str)
y_pred = y_pred.astype(str)
print(X_train.size,y_train.size)

report = classification_report(np.hstack(y_test),np.hstack(y_pred),target_names=target_names)
print(report)

(26216,)
(26216, 36)
20972 754992


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1428: UserWarning: labels size, 39, does not match size of target_names, 36
  .format(len(labels), len(target_names))


ValueError: Found input variables with inconsistent numbers of samples: [36, 188784]

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
#improve the model with grid search
pipeline = Pipeline([('scaler', StandardScaler()),('clf', SVC())])

parameters = {
    'scaler__with_mean': [True, False],
    'clf__kernel': ['linear', 'rbf'],
    'clf__C':[1, 10]
}

cv = GridSearchCV(pipeline, param_grid=parameters)

cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!



In [ ]:
### Test your model

from sklearn.preprocessing import label_binarize

# Use label_binarize to be multi-label like settings
Y = label_binarize(y, classes=[0, 1, 2])
n_classes = Y.shape[1]

# Split into training and test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.5,
                                                    random_state=random_state)

# We use OneVsRestClassifier for multi-label prediction
from sklearn.multiclass import OneVsRestClassifier

# Run classifier
classifier = OneVsRestClassifier(svm.LinearSVC(random_state=random_state))
classifier.fit(X_train, Y_train)
y_score = classifier.decision_function(X_test)


from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score

# For each class
precision = dict()
recall = dict()
average_precision = dict()
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(Y_test[:, i],
                                                        y_score[:, i])
    average_precision[i] = average_precision_score(Y_test[:, i], y_score[:, i])

# A "micro-average": quantifying score on all classes jointly
precision["micro"], recall["micro"], _ = precision_recall_curve(Y_test.ravel(),
    y_score.ravel())
average_precision["micro"] = average_precision_score(Y_test, y_score,
                                                     average="micro")
print('Average precision score, micro-averaged over all classes: {0:0.2f}'
      .format(average_precision["micro"]))

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
### Export your model as a pickle file
import pickle

# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.